In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot


In [2]:
# Add project root to Python path
project_root = Path().resolve().parents[1]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.data.db import execute_query, get_db_connection

In [3]:
# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Understanding the Data Structure

In [4]:
# Let's first understand what tables and data we have available
tables = execute_query("""
    SELECT name FROM sqlite_master
    WHERE type = 'table'
    ORDER BY name;
""")

print("Tables in database:")
for index, row in tables.iterrows():
    print(f" - {row['name']}")

Tables in database:
 - game_stats
 - pitch_mix
 - pitchers
 - prediction_features
 - sqlite_sequence
 - traditional_stats


### 1.1 Explore Each Table

In [5]:
def explore_table(table_name, limit=5):
    try:
        # Get sample data
        sample_data = execute_query(f"SELECT * FROM {table_name} LIMIT {limit}")
        print(f"\n{table_name} Table ({len(sample_data)} rows):")
        display(sample_data)
        
        # Get schema
        schema = execute_query(f"PRAGMA table_info({table_name})")
        print(f"\n{table_name} Schema:")
        display(schema)
        
        # Get row count
        count = execute_query(f"SELECT COUNT(*) as count FROM {table_name}")
        print(f"Total rows in {table_name}: {count.iloc[0]['count']}")
        
    except Exception as e:
        print(f"Error exploring {table_name}: {e}")

# Explore each table structure
for table in ['game_stats', 'pitchers', 'traditional_stats', 'pitch_mix', 'prediction_features']:
    explore_table(table)


game_stats Table (5 rows):


,id,pitcher_id,game_id,game_date,season,strikeouts,hits,walks,home_runs,release_speed_mean,release_speed_max,release_spin_rate_mean,swinging_strike_pct,called_strike_pct,zone_rate
0,1,1,661984,2022-05-15,2022,0,0,0,0,59.692593,69.6,1598.296296,0.0,0.0,0.0
1,2,2,662587,2022-04-19,2022,0,0,0,0,80.930000,88.0,2136.000000,0.0,0.0,0.0
2,3,2,662625,2022-04-17,2022,0,0,0,0,86.446154,90.6,2115.384615,0.0,0.0,0.0
3,4,2,663341,2022-04-13,2022,0,0,0,0,86.750000,86.8,2095.500000,0.0,0.0,0.0
4,5,2,663361,2022-04-24,2022,0,0,0,0,83.036364,87.9,2107.454545,0.0,0.0,0.0



game_stats Schema:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,pitcher_id,INTEGER,0,None,0
2,2,game_id,TEXT,0,None,0
3,3,game_date,TEXT,0,None,0
4,4,season,INTEGER,0,None,0
5,5,strikeouts,INTEGER,0,None,0
6,6,hits,INTEGER,0,None,0
7,7,walks,INTEGER,0,None,0
8,8,home_runs,INTEGER,0,None,0
9,9,release_speed_mean,REAL,0,None,0


Total rows in game_stats: 62264

pitchers Table (5 rows):


...

Total rows in pitch_mix: 0

prediction_features Table (5 rows):


,id,pitcher_id,game_id,game_date,season,last_3_games_strikeouts_avg,last_5_games_strikeouts_avg,last_3_games_k9_avg,last_5_games_k9_avg,last_3_games_era_avg,last_5_games_era_avg,last_3_games_fip_avg,last_5_games_fip_avg,last_3_games_velo_avg,last_5_games_velo_avg,last_3_games_swinging_strike_pct_avg,last_5_games_swinging_strike_pct_avg,days_rest,team_changed
0,1,1,661984,2022-05-15,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,5,1
1,2,2,707041,2022-04-03,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,5,1
2,3,2,663418,2022-04-07,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.090000,88.090000,0.0,0.0,4,1
3,4,2,663417,2022-04-08,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.921471,85.921471,0.0,0.0,1,1
4,5,2,663341,2022-04-13,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.428599,84.428599,0.0,0.0,5,1



prediction_features Schema:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,pitcher_id,INTEGER,0,None,0
2,2,game_id,TEXT,0,None,0
3,3,game_date,TEXT,0,None,0
4,4,season,INTEGER,0,None,0
5,5,last_3_games_strikeouts_avg,REAL,0,None,0
6,6,last_5_games_strikeouts_avg,REAL,0,None,0
7,7,last_3_games_k9_avg,REAL,0,None,0
8,8,last_5_games_k9_avg,REAL,0,None,0
9,9,last_3_games_era_avg,REAL,0,None,0


Total rows in prediction_features: 62264


## 2. Extracting Relevant Data for Analysis

In [6]:
# To analyze strikeouts, we'll need to join the relevant tables 

# Get data for analysis by joining necessary tables
pitcher_data = execute_query("""
    SELECT p.*, ts.*, pm.*, pf.*
    FROM pitchers p
    LEFT JOIN traditional_stats ts ON p.pitcher_id = ts.pitcher_id
    LEFT JOIN pitch_mix pm ON p.pitcher_id = pm.pitcher_id
    LEFT JOIN prediction_features pf ON p.pitcher_id = pf.pitcher_id
""")

# Check the shape of our joined dataset
print(f"Dataset shape: {pitcher_data.shape}")

DatabaseError: Execution failed on sql '
    SELECT p.*, ts.*, pm.*, pf.*
    FROM pitchers p
    LEFT JOIN traditional_stats ts ON p.pitcher_id = ts.pitcher_id
    LEFT JOIN pitch_mix pm ON p.pitcher_id = pm.pitcher_id
    LEFT JOIN prediction_features pf ON p.pitcher_id = pf.pitcher_id
': no such column: pm.pitcher_id